In [ ]:
import os

from itertools import product

import numpy as np

import time

import matplotlib.pyplot as plt

import sys

import statistics

import pprint

from copy import deepcopy

sys.path.append('C:\\Users\\HADDAM\\Documents\\Python Scripts\\multi_flow_decomp\\')
from msmd.multi_flow_desag_instance_utils import construct_instances
from msmd.experiments_utils import process_data_MSMD_TransF_vs_MSMD_TF_RL

os.chdir('C:\\Users\\HADDAM\\Documents\\Python Scripts\\multi_flow_decomp\\')


In [9]:
colors = ["b", "g", "r", "k", "m", "y", "orange", "brown"]
def basic_plot(
         dict_results, 
         dir_save_name, 
         id_metrics, 
         dict_titles,
         update_transport_time,
         update_transition_functions,
         legends = ["mfds", "mfds_rl_node_based","mfds_rl_arc_based"],
         data_keys = ["mfds", "mfds_rl_node_based","mfds_rl_arc_based"],
         show_ = False):
    for id_metric in id_metrics:
        fig = plt.figure()
        for i in range(len(legends)):
            title = "scatter_"+dict_titles[id_metric]+"_"+str(update_transport_time)+"_"+str(update_transition_functions)
            y_mfds = [dict_results[key][id_metric] for key in dict_results if key[1] == update_transport_time and\
                                                                                key[2] == update_transition_functions and\
                                                                                    key[3] == data_keys[i]]
            y_mfds.sort(reverse=True)
            y_mfds_rl = [dict_results[key][id_metric] for key in dict_results if key[1] == update_transport_time and\
                                                                                    key[2] == update_transition_functions and\
                                                                                        key[3] == data_keys[i]]
            y_mfds_rl.sort(reverse=True)

            plt.scatter(list(range(len(y_mfds))), y_mfds, color=colors[i], alpha=0.5, label=legends[i])
        plt.xlabel("Instances")
        plt.ylabel(dict_titles[id_metric])
        plt.legend(loc="upper right")
        fig.savefig(os.path.join(dir_save_name, title), bbox_inches='tight', pad_inches=0)
        plt.close(fig)

    if show_: plt.show()

In [10]:
save_instances = False
dict_instances = construct_instances (
                        dir_name_graph_instance = "instance_generation/instances/capacity/", 
                        dir_name_multi_flow_instance = "multi_flow_generation/transition_function_instances/",
                        nb_instances = 100,
                        ls_update_transport_time = [True],
                        ls_update_transition_functions = [True])
if save_instances: np.save("RL_methods/data/data_instances", dict_instances)

Construct instance  0
Construct instance  1
Construct instance  2
Construct instance  3
Construct instance  4
Construct instance  5
Construct instance  6
Construct instance  7
Construct instance  8
Construct instance  9
Construct instance  10
Construct instance  11
Construct instance  12
Construct instance  13
Construct instance  14
Construct instance  15
Construct instance  16
Construct instance  17
Construct instance  18
Construct instance  19
Construct instance  20
Construct instance  21
Construct instance  22
Construct instance  23
Construct instance  24
Construct instance  25
Construct instance  26
Construct instance  27
Construct instance  28
Construct instance  29
Construct instance  30
Construct instance  31
Construct instance  32
Construct instance  33
Construct instance  34
Construct instance  35
Construct instance  36
Construct instance  37
Construct instance  38
Construct instance  39
Construct instance  40
Construct instance  41
Construct instance  42
Construct instance  4

In [12]:
# Test names
test_names = {"basic_comp_MSMD_TransF_vs_MSMD_TF_RL",
              "search_learning_rate"}
test_name = "basic_comp_MSMD_TransF_vs_MSMD_TF_RL"

if test_name not in test_names:
        print("Test name or test algorithm unrecognized.")
        sys.exit()

if test_name == "basic_comp_MSMD_TransF_vs_MSMD_TF_RL":
        dict_instances = np.load("RL_methods/data/data_instances.npy", allow_pickle = True).flatten()[0]
        dict_params_rl_agent = {"ag_type":"LRI",
                                "lr":0.05,
                                "eps":None,
                                "opt_params":{"initial_actions_estimates":None}}
        """to_be_tested_list = ["mfds_min_time", "mfds_max_capacity", 
                             "mfds_random_path", "mfds",
                             "mfds_support_path"]"""
        to_be_tested_list = ["mfds"]
        dict_results = process_data_MSMD_TransF_vs_MSMD_TF_RL (
                                     dict_instances,
                                     dict_params_rl_agent = dict_params_rl_agent,
                                     nb_episodes = 5,
                                     coeff1 = 0.33, coeff2 = 0.33, coeff3 = 0.34, # 0.33, 0.33, 0.34
                                     update_transport_time = True,
                                     update_transition_functions = True,
                                     pair_criteria = "max_remaining_flow_val",
                                     path_card_criteria = "one_only", # "one_for_each" "one_only"
                                     maximal_flow_amount = 1,
                                     nb_max_tries = 50000,
                                     max_path_length = 10000,
                                     max_nb_tries_find_path = 10,
                                     reodering_pairs_policy_name = None, # None "remaining_max_flow"
                                     exclude_chosen_nodes = False,
                                     successor_selector_type = "exponential_decay",
                                     rl_data_init_type = "uniform",
                                     to_be_tested_list = to_be_tested_list,
                                     penalty_chosen_path = True,
                                     save_RL_dir = None,
                                     override_update_transition_function = [],
                                     ignore_conflicts = False,
                                     exit_on_none = False)
        np.save("results/simulated/MFDS_vs_RL/results.npy", dict_results)
        dict_results = np.load("results/simulated/MFDS_vs_RL/results.npy", allow_pickle = True).flatten()[0]
        basic_plot(
                dict_results, 
                "results/simulated/MFDS_vs_RL/", 
                id_metrics = [0, 1, 2, 5], 
                dict_titles = {0:"flow_val_res", 
                                1:"flow_res", 
                                2:"m_flow_res", 
                                3:"prop_fsupp", 
                                4:"prop_sp", 
                                5:"trans_func_res"},
                update_transport_time = True,
                update_transition_functions = True,
                legends = to_be_tested_list,
                data_keys = to_be_tested_list,
                show_ = False)

elif test_name == "search_learning_rate":
       learning_rate_ls = [0.001, 0.005, 0.01, 0.05, 0.1, 0.2] 

Case  0 True True mfds
Number of iteration  442
Proportion of support arcs   0.9921875
Proportion of flow val residue  0.17996289424860853
Proportion of flow residue   0.16177317213586645
Proportion of multi flow residue   0.2040875071963155
Proportion of trans func residue   0.15630397236614854
Original flow values  [47, 39, 23, 29, 23, 64, 18, 36, 23, 72, 51, 20, 43, 35, 16]
Desaggregated Flow values  [41, 30, 20, 25, 12, 55, 11, 29, 13, 67, 48, 11, 37, 32, 11]
Case  1 True True mfds
Number of iteration  429
Proportion of support arcs   0.9780701754385965
Proportion of flow val residue  0.15217391304347827
Proportion of flow residue   0.14025623735670936
Proportion of multi flow residue   0.17666891436277815
Proportion of trans func residue   0.124409979770735
Original flow values  [39, 41, 65, 39, 21, 7, 40, 50, 49, 33, 24, 10, 31, 27, 30]
Desaggregated Flow values  [38, 34, 56, 39, 15, 3, 37, 42, 47, 24, 18, 4, 29, 20, 23]
Case  2 True True mfds
Number of iteration  499
Proportion 

KeyboardInterrupt: 

In [ ]:
dir_dict_res = "results/simulated/MFDS_vs_RL/arc_based/eps=100_lr=0,05/"
file_res = "results.npy"
data_dict_res = np.load(dir_dict_res+file_res, allow_pickle = True).flatten()[0]
upd_transport_time, upd_transition_function = True, True
# dict_result[(ind_instance, upd_transport_time, upd_transition_function, test_)] = (flow_val_res, flow_res, m_flow_res, prop_fsupp, prop_sp, trans_func_res)
msmd_tf = [(key[0], data_dict_res[key][2]) for key in data_dict_res if key[1] == upd_transport_time and\
                                                                            key[2] == upd_transition_function and\
                                                                                key[3] == 0]
msmd_tf.sort(key = lambda e : -e[1])
msmd_tf_rl = [(key[0], data_dict_res[key][2]) for key in data_dict_res if key[1] == upd_transport_time and\
                                                                            key[2] == upd_transition_function and\
                                                                                key[3] == 1]
msmd_tf_rl = [data_dict_res[(elem[0], upd_transport_time, upd_transition_function, 1)][2] for elem in msmd_tf]
msmd_tf = [e[1] for e in msmd_tf]

fig = plt.figure()
plt.scatter(list(range(len(msmd_tf))), msmd_tf, color="b", alpha=0.5, label="mfds")
plt.scatter(list(range(len(msmd_tf_rl))), msmd_tf_rl, color="r", alpha=0.5, label="mfds_rl")
plt.xlabel("Instances")
plt.ylabel("m_flow_res")
plt.legend(loc="upper right")
fig.savefig(os.path.join("results/simulated/MFDS_vs_RL/", "comp_trie"), bbox_inches='tight', pad_inches=0)
plt.close(fig)
    


In [9]:
dir_save_name = "results/simulated/MFDS_vs_RL/"
fl_dict_res_penal = "results/simulated/MFDS_vs_RL/comp_transf_stand_rlnode_rlarc/penalty_chosen_path=True/results.npy"
fl_dict_res_nopenal = "results/simulated/MFDS_vs_RL/comp_transf_stand_rlnode_rlarc/penalty_chosen_path=False/results.npy"
dict_res_penal = np.load(fl_dict_res_penal, allow_pickle = True).flatten()[0]
dict_res_nopenal = np.load(fl_dict_res_nopenal, allow_pickle = True).flatten()[0]
msmd_penal = [dict_res_penal[key][2] for key in dict_res_penal if key[1] == True and\
                                                                    key[2] == True and\
                                                                        key[3] == "mfds_rl_arc_based"]
msmd_penal.sort(reverse=True)
print("msmd_penal ", np.mean(msmd_penal), " +/- ", np.std(msmd_penal))
msmd_nopenal = [dict_res_nopenal[key][2] for key in dict_res_nopenal if key[1] == True and\
                                                                            key[2] == True and\
                                                                                key[3] == "mfds_rl_arc_based"]
msmd_nopenal.sort(reverse=True)
print("msmd_nopenal ", np.mean(msmd_nopenal), " +/- ", np.std(msmd_nopenal))
fig = plt.figure()
plt.scatter(list(range(len(msmd_penal))), msmd_penal, color="r", alpha=0.5, label="msmd_penal")
plt.scatter(list(range(len(msmd_nopenal))), msmd_nopenal, color="b", alpha=0.5, label="msmd_nopenal")
plt.xlabel("Instances")
plt.ylabel("m_flow_res")
plt.legend(loc="upper right")
fig.savefig(os.path.join(dir_save_name, "courbe"), bbox_inches='tight', pad_inches=0)
plt.close(fig)

msmd_penal  0.06473558291107825  +/-  0.02992656672900103
msmd_nopenal  0.06872574981877515  +/-  0.031828387255019894
